In [44]:
import pandas as pd
import numpy as np
import pokebase as pb
import os

In [45]:
# helper function to unpack evolution chain

def get_evolutions(species_name, chain_node):
    # check if the current node is the one we're looking for
    if chain_node.species.name == species_name.lower():
        return [evo.species.name.title() for evo in chain_node.evolves_to]

    # otherwise, search recursively through the evolves_to list
    for next_node in chain_node.evolves_to:
        result = get_evolutions(species_name, next_node)
        if result is not None:
            return result
            
    return None

In [46]:
# helper function to extract pokemon data into dictionary

def pokemon_to_dict(pkmn_id):
    pkmn = pb.pokemon(pkmn_id)
    pkmn_species = pkmn.species
    pkmn_chain = pkmn_species.evolution_chain.chain

    forms_list = []
    
    # pokemon have varities like megas and we want those stats as well
    # loop through any additional varities and append those to list
    for variety in pkmn_species.varieties:
        form_data = variety.pokemon
        
        pkmn_dict = {
            'id ': form_data.id, # Keep the ID the same for all varieties of this species
            'name': form_data.name.title(), # e.g. 'Charizard-Mega-X'
            'primary_type': form_data.types[0].type.name.capitalize(),
            'secondary_type': form_data.types[1].type.name.capitalize() if len(form_data.types) > 1 else np.nan,
            'hp': form_data.stats[0].base_stat,
            'attack': form_data.stats[1].base_stat,
            'defense': form_data.stats[2].base_stat,
            'special_attack': form_data.stats[3].base_stat,
            'special_defense': form_data.stats[4].base_stat,
            'speed': form_data.stats[5].base_stat,
            'evolution': get_evolutions(pkmn_species.name, pkmn_chain),
            'is_baby': pkmn_species.is_baby,
            'is_legendary': pkmn_species.is_legendary,
            'is_mythical': pkmn_species.is_mythical,
            'is_mega': 'mega' in form_data.name.lower(),
            'generation': pkmn_species.generation.name,
            'species': pkmn_species.name.title()
        }
        forms_list.append(pkmn_dict)
        
    # Return the list of dictionaries
    return forms_list

In [47]:
full_pokedex = []

# Because there are finite amount of pokemon and they change so infrequently, we will hardcode the known range to grab
for id in range(1,1026):
    pkmn_forms_list = pokemon_to_dict(id)
    # extend handles the list of dictionaries and unpacks them
    full_pokedex.extend(pkmn_forms_list)

pokedex_df = pd.DataFrame(full_pokedex)
# Let's convert this df to a csv given the time it took to pull, then create a new notebook to work off that csv

home_dir = os.path.expanduser('~')
download_folder = 'Downloads'
downloads_path = os.path.join(home_dir, download_folder)

csv_name = 'nat_pokedex_data.csv'

full_file_path = os.path.join(downloads_path, csv_name)
pokedex_df.to_csv(full_file_path, index=False)
